In [ ]:
%matplotlib inline
from collections import OrderedDict
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import math
import os
from scipy.optimize import brentq
from sklearn.neighbors import KernelDensity
import xarray as xr

from filterpy.kalman import UnscentedKalmanFilter as UKF
from filterpy.kalman import KalmanFilter as KF
from filterpy.kalman import MerweScaledSigmaPoints
from filterpy.common import Q_continuous_white_noise

from pumahu import Forwardmodel, LakeData, get_data
from pumahu.visualise import trellis_plot, TrellisPlot, heat_vs_rsam, plot_qin_uks
from pumahu.uks import UnscentedKalmanSmoother
from pumahu.fd import fd

In [ ]:
def xr2xr(xdfin):
    """
    Helper function to convert old netcdf file structure to new
    """
    new_exp = np.zeros((xdfin['exp'].shape[0], xdfin['exp'].shape[1], 2))
    new_exp[:, :, 0] = xdfin['exp'].values[:]
    new_exp[:, :, 1] = xdfin['var'].values[:]

    return xr.Dataset({'exp': (('dates', 'parameters', 'val_std'), new_exp)},
                      {'dates': xdfin.dates.values,
                       'parameters': xdfin.parameters.values,
                       'val_std': ['val', 'std']})

## Table of contents
---------------------------
1. [Inversion](#Inversion)
2. [Forecasting](#Forecasting)

## Inversion

In [ ]:
enddate = datetime(2021,12,18)
startdate = enddate - timedelta(days=30)
res_dir = './data'
if not os.path.isdir(res_dir):
    os.makedirs(res_dir)
mout_prior = get_data('./data/outflow_prior.npz')

In [ ]:
ld = LakeData()
data = ld.get_data(startdate, enddate)

In [ ]:
data_uks = ld.get_data(startdate, enddate, smoothing='dv')

In [ ]:
uks4 = UnscentedKalmanSmoother(data=data)
xds_uks = uks4()

In [ ]:
plot_qin_uks(xds_uks, annotations=True)

In [ ]:
xds_fd = fd(data, use_drmg=True)

In [ ]:
trellis_plot(xds_fd, data2=xds_uks)

In [ ]:
_ = heat_vs_rsam(xds_uks)

In [ ]:
#benchmark_file = './data/benchmarking/forward_2019-01-01_2021-01-11.nc'
benchmark_file = None
xdft = None
if benchmark_file is not None:
    xdft = xr2xr(xr.open_dataset(benchmark_file))

## Run the estimate for a whole year

In [ ]:
runyear = False

In [ ]:
if runyear:
    startdate_year = datetime.utcnow()-timedelta(days=365)
    enddate_year = datetime.utcnow()
    ld = LakeData()

In [ ]:
if runyear:
    data_uks_year = ld.get_data(startdate_year, enddate_year, smoothing='dv')
    Q = OrderedDict(T=1e-1, M=1e1, X=1e-3, q_in=1e2,
                    m_in=1e1, m_out=1e1, h=1e-3, W=1e-3,
                    dqi=0, dMi=0, dMo=0, dH=0, 
                    dW=0)
    Q = np.eye(len(Q))*list(Q.values())
    uks4 = UnscentedKalmanSmoother(data=data_uks_year, Q=Q)
    xds_uks_year = uks4()

In [ ]:
if runyear:
    fig = trellis_plot(xds_uks_year)
    fig.show()